In [1]:
import os
import numpy as np
from stl import mesh
from mpl_toolkits import mplot3d
import matplotlib.pyplot as plt
from scipy.spatial import ConvexHull, convex_hull_plot_2d
import vtkplotlib as vpl
%matplotlib inline

In [2]:

objFilePath = 'meshes\\spheres.obj'

points = []
faces = []
with open(objFilePath) as file:
    while 1:
        line = file.readline()
        if not line:
            break
        strs = line.split(" ")
        if strs[0] == "v":
            points.append((float(strs[1]), float(strs[2]), float(strs[3])))
        if strs[0] == "f":
            faces.append((int(strs[1]), int(strs[2]), int(strs[3])))
             
points = np.array(points)
faces = np.array(faces)

In [3]:
points

array([[ 0.000000e+00,  1.000000e+00,  0.000000e+00],
       [ 6.279100e-02,  9.980270e-01,  0.000000e+00],
       [ 1.574200e-02,  9.995010e-01, -1.989000e-03],
       ...,
       [ 1.098300e-01,  1.007885e+00, -2.060380e+00],
       [ 1.165320e-01,  1.007885e+00, -2.046138e+00],
       [ 1.213960e-01,  1.007885e+00, -2.031169e+00]])

In [4]:
np.amin(faces)

1

In [5]:
your_mesh = mesh.Mesh.from_file('make-it-stand_data\models\\armadillo1\\armadillo1_outer.stl')


In [6]:
# The mesh normals (calculated automatically)
your_mesh.normals
# The mesh vectors
your_mesh.v0, your_mesh.v1, your_mesh.v2
# Accessing individual points (concatenation of v0, v1 and v2 in triplets)
assert (your_mesh.points[0][0:3] == your_mesh.v0[0]).all()
assert (your_mesh.points[0][3:6] == your_mesh.v1[0]).all()
assert (your_mesh.points[0][6:9] == your_mesh.v2[0]).all()
assert (your_mesh.points[1][0:3] == your_mesh.v0[1]).all()

In [7]:
vpl.mesh_plot(your_mesh)

# Show the figure
vpl.show()

In [8]:
volume, cog, inertia = your_mesh.get_mass_properties()
print("Volume                                  = {0}".format(volume))
print("Position of the center of gravity (COG) = {0}".format(cog))
print("Inertia matrix at expressed at the COG  = {0}".format(inertia[0,:]))
print("                                          {0}".format(inertia[1,:]))
print("                                          {0}".format(inertia[2,:]))

Volume                                  = 0.030976354226644975
Position of the center of gravity (COG) = [0.69577698 0.50558633 0.6387259 ]
Inertia matrix at expressed at the COG  = [1.12050744e-03 1.65343020e-05 2.73965520e-04]
                                          [1.65343020e-05 1.16590708e-03 1.55478062e-04]
                                          [0.00027397 0.00015548 0.00126509]


In [9]:
def intersect(face):
    l = []
    for i in range(len(face)):
        l.append(points[face[i]-1]) #我这里的faces里面的points序号从1开始 需要减1
    res = []
    l.append(l[0])
    #print(l)
    
    for i in range(3):
        p1 = l[i]
        p2 = l[i+1]
        #print(p1,p2)
        x1,y1,z1 = p1
        x2,y2,z2 = p2
        
        k=0
        if z2 - z1 != 0:
            k = (-z1)/(z2-z1)
            x = k*(x2-x1)+x1
            y = k*(y2-y1)+y1

            if (x1<x<x2 or x2<x<x1) and (y1<y<y2 or y2<y<y1):
                res.append([x,y,0])
        
    return res

In [10]:
intersect(faces[0])

[]

In [11]:
faces.shape

(12080, 3)

In [12]:
poly_points = []
for face in list(faces):
    l = intersect(face)

    poly_points = poly_points + l
print(len(poly_points))

0


In [13]:
polygon = []
for p in poly_points:
    polygon.append([p[0],p[1]])
polygon = np.array(polygon)
print(np.array(polygon).shape)

(0,)


In [14]:
hull = ConvexHull(polygon)
plt.plot(polygon[:,0], polygon[:,1], 'o')
for simplex in hull.simplices:
    plt.plot(polygon[simplex, 0], polygon[simplex, 1], 'k-')

IndexError: tuple index out of range

In [147]:
def cut(num_face):
    l = []
    face = faces[num_face].copy()
    for i in range(len(face)):
        l.append(points[face[i]-1]) #我这里的faces里面的points序号从1开始 需要减1
    res = []
    l.append(l[0])
    #print(l)
    
    for i in range(3):
        #print(i,l)
        p1 = l[i]
        p2 = l[i+1]
        #print(p1,p2)
        x1,y1,z1 = p1
        x2,y2,z2 = p2
        
    
        if abs(z2-z1) > 0.01:
            k = (-z1)/(z2-z1)
            x = k*(x2-x1)+x1
            y = k*(y2-y1)+y1

            if (x1<x<x2 or x2<x<x1) and (y1<y<y2 or y2<y<y1):
                res.append([x,y,0])
            
    n = len(res)
    
    #三角形全在下面 删去
    if n == 0:
        if (l[0][2]<0 and l[1][2]<0 and l[2][2]<0):
            del_face.append(num_face)
            #faces.pop(num_face)
            return []
        
    #有部分点在下面 删去
    for i in range(3):
        if l[i][2]<0:
#             print(face)
#             print(l)
#             print(faces[num_face])
            faces[num_face].remove(face[i])
    
            
    #判断新点是否已经加入 
    l_add = []
    for i in range(len(res)):
        js = -1
        p_add = res[i]
        for j in range(len(points)):
            if points[j] == p_add:
                js = j
                break
        l_add.append(js)
    if (len(l_add)>2):
        print('l_add ',len(l_add),l_add)
    
    #没有 加入
    for i in range(len(l_add)):
        if l_add[i] == -1:
            l_add[i] = len(points)
            points.append(res[i])
                
    #更新face里的点
    if (len(l_add)==1):
        print('l_add ',len(l_add),l_add,res,l)
    if (len(faces[num_face])+len(l_add)>4):
        print('faces[num_face] ',len(faces[num_face]))
        print(l,'\n',res)
    for i in range(len(l_add)):
        faces[num_face].append(l_add[i])            
        
    return l_add

In [148]:
objFilePath = 'meshes\\armadillo1.obj'

points = []
faces = []
with open(objFilePath) as file:
    while 1:
        line = file.readline()
        if not line:
            break
        strs = line.split(" ")
        if strs[0] == "v":
            points.append((float(strs[1]), float(strs[2]), float(strs[3])))
        if strs[0] == "f":
            faces.append((int(strs[1]), int(strs[2]), int(strs[3])))
             
points = np.array(points)
faces = np.array(faces)

In [149]:
points = points.tolist()
faces = faces.tolist()

In [150]:
for i in range(len(points)):
    points[i][2] -= 0.0001

In [151]:
for i in range(len(faces)):
    if len(faces[i]) > 3:
        print(faces[i])

In [152]:
i=0
print(faces[i])
for j in faces[i]:
    print(points[j])

[2, 1, 23172]
[-0.851321, -0.078791, 0.033755999999999994]
[-0.136929, 0.15039, -0.093123]
[-0.386323, -0.117418, -0.19111599999999998]


In [153]:
cut_points = []
del_face = []
for i in range(len(faces)):
    l = cut(i)
    if (len(l)>0):
        cut_points .append(l) 
print(len(cut_points))

l_add  1 [43205] [[-0.3347660444483028, 0.2319889453945655, 0]] [[-0.332074, 0.233418, 0.01034], [-0.335073, 0.231826, -0.0011790000000000001], [-0.340316, 0.234452, 0.007331], [-0.332074, 0.233418, 0.01034]]
l_add  1 [43206] [[-0.011653478776185226, -0.14765825413450936, 0]] [[0.000758, -0.141347, -0.010683], [-0.009948, -0.135048, -0.0039], [-0.011887, -0.147777, 0.000201], [0.000758, -0.141347, -0.010683]]
l_add  1 [43206] [[-0.011653478776185226, -0.14765825413450936, 0]] [[0.000758, -0.141347, -0.010683], [-0.011887, -0.147777, 0.000201], [-0.002892, -0.155842, -0.005030000000000001], [0.000758, -0.141347, -0.010683]]
l_add  1 [43207] [[0.0070804075847538785, 0.22908483384409117, 0]] [[0.011409, 0.225143, 0.001163], [0.006805, 0.228243, -0.002433], [0.007987, 0.231856, 0.008009], [0.011409, 0.225143, 0.001163]]
l_add  1 [43207] [[0.0070804075847538785, 0.22908483384409117, 0]] [[0.001402, 0.234426, -0.0012720000000000001], [0.007987, 0.231856, 0.008009], [0.006805, 0.228243, -0.00

l_add  1 [43227] [[-0.14422936408929768, -0.6101145849382007, 0]] [[-0.144913, -0.611369, 0.008474], [-0.144071, -0.609824, -0.001963], [-0.139682, -0.607677, 0.003428], [-0.144913, -0.611369, 0.008474]]
l_add  1 [43228] [[-0.13317625749368, -0.5868187522571325, 0]] [[-0.134084, -0.593982, 0.0038500000000000006], [-0.133008, -0.586637, -0.000473], [-0.136948, -0.590893, 0.010603000000000001], [-0.134084, -0.593982, 0.0038500000000000006]]
l_add  1 [43228] [[-0.13317625749368, -0.5868187522571325, 0]] [[-0.137183, -0.582736, 0.006704], [-0.136948, -0.590893, 0.010603000000000001], [-0.133008, -0.586637, -0.000473], [-0.137183, -0.582736, 0.006704]]
l_add  1 [43229] [[-0.2079504378796245, -0.5791096968525676, 0]] [[-0.205535, -0.573136, -0.002297], [-0.208172, -0.576845, -0.007842], [-0.207865, -0.579983, 0.003024], [-0.205535, -0.573136, -0.002297]]
l_add  1 [43229] [[-0.2079504378796245, -0.5791096968525676, 0]] [[-0.207865, -0.579983, 0.003024], [-0.208172, -0.576845, -0.007842], [-0.

In [145]:
for i in range(len(faces)):
    if len(faces[i]) < 3:
        del_face.append(i)

In [127]:
del_face.sort(reverse=True)
print(len(del_face),len(faces))

56806 86406


In [128]:
for num_face in del_face:
    faces.pop(num_face)

In [129]:
for i in range(len(faces)):
    if len(faces[i]) == 4:
        faces.append([faces[i][0],faces[i][1],faces[i][3]])
        faces[i].pop(3)

In [130]:
for i in range(len(faces)):
    if len(faces[i]) > 3:
        print(faces[i])

In [131]:
def test_obj():
    thefile = open('test.obj', 'w')
    for item in points:
        thefile.write("v {0} {1} {2}\n".format(item[0],item[1],item[2]))

    for item in faces:
        thefile.write("f {0} {1} {2}\n".format(item[0],item[1],item[2]))  

    thefile.close()
    return

In [132]:
test_obj()

In [133]:
c_cp = cut_points.copy()

In [134]:
cut_points = c_cp.copy()

In [135]:
cut_points[:5]

[[43205, 43206],
 [43205, 43207],
 [43207, 43208],
 [43209, 43210],
 [43210, 43208]]

In [137]:
for i in range(len(cut_points)):
    if len(cut_points[i])!=2:
        print(cut_points[i])

[43205, 43206, 43207, 43208, 43210, 43215, 43231, 43232, 43235, 43237, 43236]
[44536]
[44537]


In [136]:
new_face = [cut_points[0]]
while len(cut_points)>0:
    del_l = []
    for i in range(len(cut_points)):
        p1,p2 = cut_points[i]
        for j in range(len(new_face)):
            if (p1 in new_face[j]) or (p2 in new_face[j]):
                if not (p1 in new_face[j]):
                    new_face[j].append(p1)
                if not (p2 in new_face[j]):
                    new_face[j].append(p2)
                del_l.append(i)
    if len(del_l) == 0:
        new_face.append(cut_points[0])
        cut_points.pop(0)
    else:
        del_l.sort(reverse=True)
        for i in del_l:
            cut_points.pop(i)
    
    
                

ValueError: not enough values to unpack (expected 2, got 1)

In [83]:
print(len(new_face))
print(new_face[0][:5])

1
[6042]


In [84]:
n = len(new_face)

for i in range(n):
    for j in range(i+1,n):
        set1=new_face[i]
        set2=new_face[j]
        x = [k for k in set1 if k in set2]
        if len(x)>0:
            print(i,' ',j,' ',list(x))

In [34]:
for face in new_face:
    faces.append(face)

NameError: name 'new_face' is not defined

In [35]:
test_obj()